In [77]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.externals import joblib
import sklearn

In [113]:
test_data_cols1 = pd.read_csv('test_data_cols1.csv', sep=',')
test_data_cols2 = pd.read_csv('test_data_cols2.csv', sep=',', skiprows=1, header=None)

In [114]:
# Creating DF with features and duplicate variable only for final model
def append_df(df1, df2):
    # df1 with column names, and df2 without
    df2.columns = df1.columns
    df1 = df1.append(df2, ignore_index=True)
    return df1

In [115]:
test_data_cols1 = append_df(test_data_cols1,test_data_cols2) 

In [116]:
test_data_cols1.count()

test_id            2345796
q1_form_len        2345796
q2_form_len        2345796
q1_length          2345796
q2_length          2345796
q1_unique          2345796
q2_unique          2345796
q1_form_uni        2345796
q2_form_uni        2345796
q1_form_char       2345796
q2_form_char       2345796
q1_q2_char_diff    2345796
common_cnt         2345796
prcnt_common       2345379
dtype: int64

In [117]:
test_data_cols1['prcnt_common'].fillna(test_data_cols1['prcnt_common'].mean(), inplace=True)

In [118]:
test_data_cols1.count()

test_id            2345796
q1_form_len        2345796
q2_form_len        2345796
q1_length          2345796
q2_length          2345796
q1_unique          2345796
q2_unique          2345796
q1_form_uni        2345796
q2_form_uni        2345796
q1_form_char       2345796
q2_form_char       2345796
q1_q2_char_diff    2345796
common_cnt         2345796
prcnt_common       2345796
dtype: int64

## Test Score

In [119]:
def test_clean(test_data):
    # Create Test_ID DF
    test_id_col = test_data["test_id"]
    test_id_col = pd.DataFrame(pd.Series(test_id_col))
    
    # Clean Test DF model
    test_data.drop(['test_id'], axis=1, inplace=True)
    #test_data.drop(['prcnt_common'], axis=1, inplace=True)               # Include this column in the main version-Comment it
    
    return test_id_col, test_data

In [120]:
test_id_col, test_data_cols1 = test_clean(test_data_cols1)

In [121]:
def test_df_create(model_path, model_name, test_data, test_id_col, submission_path, final_file_name):
    # Loading the Desired Model
    xgb2 = joblib.load(model_path+model_name)
    
    # Run model on Test Data
    test_pred_xgb2 = xgb2.predict(test_data)
    test_pred_xgb2 = pd.DataFrame(test_pred_xgb2)
    test_pred_xgb2.columns=['is_duplicate']
    
    # Final Submission DF
    submit_df = pd.concat([test_id_col,test_pred_xgb2 ], axis=1)
    submit_df.to_csv(test_submission_path+final_file_name, index=False, sep=',')
    
    # Basis info
    print "Test DF Shape",(submit_df.shape[0], submit_df.shape[1])
    print(submit_df.head())
    print('Total 1s : %d' % (test_pred_xgb2.sum()))
    print('1s Percentage : %f' % (float(test_pred_xgb2.sum())/float(submit_df.shape[0])*100))
    
    return(submit_df)

## Scoring on Model of Basic Features 

In [122]:
model_path='/home/pgoyal/quora/model/'
test_submission_path= '/home/pgoyal/quora/'

In [112]:
# Model-1 : Choose this one only as of now (Score 8.69~) : Model without prcnt_common feature
model_name= 'xgb2_dup_auc.pkl'
file_name = 'test_submission_1.csv'
final_df = test_df_create(model_path, model_name, test_data_cols1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             1
3        3             0
4        4             0
Total 1s : 356786
1s Percentage : 15.209592


In [123]:
# Model-2 : Not Chosen (Score 14.~)  : Model with prcnt_common
model_name= 'xgb2_dup_auc_2.pkl'
file_name = 'test_submission_2.csv'
final_df = test_df_create(model_path, model_name, test_data_cols1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             1
3        3             0
4        4             0
Total 1s : 928668
1s Percentage : 39.588609


## Scoring on Model of Basic+Fuzzy Features